In [1]:
#####################This program uses Selenium which has known issues of intermittance#############
############################Execute all cells in sequence#########################################
################Don't start executing next cell till execution of one is finished##################
################Don't give "Restart and Run All" command from Kernel above#########################

# import necessary libraries
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Install webdriver manager to automatically detect driver location
service = ChromeService(executable_path=ChromeDriverManager().install())

# Set up ChromeDriver options
chrome_options = Options()
chrome_options.add_argument('--headless') # Run in headless mode to avoid opening a new browser window

# Initialize empty list to store results
all_hrefs = []

n = 5 #Number of list pages to be looped over

# Loop over 5 list pages
for i in range(1,n+1):
    # Build URL for current list pages having links to company profiles
    url = f"https://wirmarket.wir.ch/de/members/list/?page={i}&resultAmount=100"

    # Launch ChromeDriver and load websites
    # In order to disable browser windows opening for websites amend the code of the following line:
    # driver = webdriver.Chrome(service=service, options = chrome_options)
    driver = webdriver.Chrome(service=service)
    
    # Don't uncomment the code on the following line, it is for testing
    #   driver.implicitly_wait(20) # seconds
    driver.get(url)

    # Wait for all anchor elements to be present
    # WebDriverWait(driver, 180).until(EC.visiblity_of_all_elements_located((By.CSS_SELECTOR, "a")))
    # Due to intermittent functionality of WebDriverWait, we introduce sleep() function to get all anchors
    # This value is presently set at 15 secs but you may change it depending upon internet speed
    time.sleep(25)
    # Extract hrefs from all anchor elements on current website
    hrefs = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, "a")]

    # Add hrefs to the list of all hrefs
    all_hrefs.extend(hrefs)

    # Close the current browser window
    driver.quit()

# Print all hrefs
# print(all_hrefs)


[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.78M/6.78M [00:00<00:00, 8.04MB/s]


In [2]:
# Check the length of all_hrefs list
# Each list link contains between 400-500 hrefs
print(len(all_hrefs))
# Check for duplicates
print(len(set(all_hrefs)))

2344
1050


In [3]:
# Filter the list 'all_hrefs' to keep only the company profile page urls

#import regex library
import re
# regex pattern for company profile pages
pattern = r'https:\/\/\w+\.wir\.ch\/de\/companyProfile\/profile\/[0-9A-F]{32}\/info\/\?promo=false$'

company_profile_urls = []

# loop over all_hrefs list extracted above with Selenium to extract company profile urls
for href in all_hrefs:
    match = re.search(pattern, str(href))
    if match:
        url = (match.group())
        company_profile_urls.append(url)

In [4]:
# check the length of the list "coompany_profiles_urls"
print(len(company_profile_urls))
# extract unique urls
company_profile_set = set(company_profile_urls)
print(len(set(company_profile_urls))) 
# change back to list
company_profile_urls = list(company_profile_set)
print(len(company_profile_urls))
print(company_profile_urls[:10])

1497
495
495
['https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF26DB2A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF429C2A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF1B502A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/48B16CDC18A536AEE054A0369F14B95F/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF17512A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF185B2A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF3A8A2A65E0540010E0244DC9/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/4A43CC3BB86D3E84E054A0369F14B95F/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfile/profile/58F4AE743D3F4F79E05400144FF95A47/info/?promo=false', 'https://wirmarket.wir.ch/de/companyProfi

In [ ]:
# # Get website links of all company websites available on the company profile page as href
# # We will need to start the Selenium WebDriver again

# pattern = r'^https?:\/\/(?:www\.)?([^:\/\n]+)(?:\/)?$'
# # specify the URL of the website that you want to scrape
# all_profile_hrefs = []
# for company_profile_url in company_profile_urls:
    
#     # Extract hrefs from all anchor elements on current website
#     profile_hrefs = [a.get_attribute('href') for a in driver.find_elements(By.TAG_NAME, "a")]
#     for href  in profile_hrefs:
#         match = re.search(pattern, str(href))
#         if match:
#             url = (match.group())
#     # Add hrefs to the list of al_profile_hrefs
#             all_profile_hrefs.append(url)


In [ ]:
print(len(all_profile_hrefs))

In [ ]:
print(all_profile_hrefs)

In [ ]:
# Script to extract email from a list of websites

import requests
import re
websites = all_profile_hrefs
# websites = ["https://www.colorino.ch/",
#             "https://www.easywash.ch/home",
#             "https://www.pneuboesiger.ch/",
#            'https://www.arsmusica.com/',
#             'http://www.auto-moersburg.ch',
#             'http://www.prodemo.com', 
#            ' http://www.tawan-thai.ch',
#            ' http://www.silvretta.ch']
website_emails = {}

for website in websites:
    response = requests.get(website)
    if response.status_code == 200:
        # regex pattern to match email addresses
        pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        # find all email addresses in the website content
        emails = re.findall(pattern, response.text)
        if emails:
            print(f"Email addresses found on {website}:")
        else:
            print(f"email information not present on {website}")
        
        
        for email in emails:
            website_emails[website] = email
            print(email)
    else:
        print(f"Failed to retrieve content from {website}")


In [53]:
# # pattern = r'^https?:\/\/(?:www\.)?([^:\/\n]+)(?:\/)?$' regex pattern is not needed here as we have the ids
# from contextlib import suppress

# company_names = []
# company_websites = []
# company_telephones = []
# company_fax = []
# confirm_websites = []

# for url in company_profile_urls:
#     # We put the entire code-block inside the suppress function to ignore known exceptions as try-except is overridden by Selenium
#     with suppress(Exception):
#         driver = webdriver.Chrome(service=service)
#         driver.get(url)
# #     time.sleep(3) #activate sleep if WebDriverWait behaves interittently
    
#         WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "68-lnk")))
#         WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "66-lnk")))
#         WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "fax-lnk")))
#         WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "71-lnk")))
    
#         # Get company name and website
#         anchor = driver.find_element(By.ID,"71-lnk")
#         company_name = anchor.text
#         company_names.append(company_name)   
#         company_website = anchor.get_attribute('href')
    
#         # Confirm website
#         anchor = driver.find_element(By.ID, "68-lnk")
#         confirm_website = anchor.get_attribute('href')
#         confirm_websites.append(confirm_website)
    
#         # Get company telephone number
#         anchor = driver.find_element(By.ID, "66-lnk")
#         company_telephone = anchor.get_attribute('href')
#         company_telephones.append(company_telephone)
    
#         # Get company fax number
#         anchor = driver.find_element(By.ID, "fax-lnk")
#         company_fax_no = anchor.get_attribute('href')
#         company_fax.append(company_fax_no)
    
#         driver.quit()
    
    
    


In [78]:
# from contextlib import suppress

# company_names = []
# company_websites = []
# company_telephones = []
# company_fax = []
# confirm_websites = []

# test_urls = ['https://wirmarket.wir.ch/de/companyProfile/profile/4F9AD522575A2205E054A0369F14B95F/info/?promo=false',
#             'https://wirmarket.wir.ch/de/companyProfile/profile/4F9AD522575A2205E054A0369F14B95F/info/?promo=false',
#             'https://wirmarket.wir.ch/de/companyProfile/profile/3E429FAF53F52A65E0540010E0244DC9/info/?promo=false']
# for url in test_urls:
# # for url in company_profile_urls:
#     driver = webdriver.Chrome(service=service)
#     driver.get(url)

# #     with suppress(Exception): 
# #         try:
# #             WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "68-lnk")))
# #             anchor = driver.find_element(By.ID, "68-lnk")
# #             confirm_website = anchor.get_attribute('href')
# #         except (NoSuchElementException, TimeoutException):
# #             confirm_website = ''
# #             confirm_websites.append(confirm_website)
# #         finally:
# #                 pass
    

# #         try:
# #             WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "66-lnk")))
# #             anchor = driver.find_element(By.ID, "66-lnk")
# #             company_telephone = anchor.get_attribute('href')
# #         except (NoSuchElementException, TimeoutException):
# #             company_telephone = ''
# #             company_telephones.append(company_telephone)
# #         finally:
# #                 pass

# #         try:
# #             WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "fax-lnk")))
# #             anchor = driver.find_element(By.ID, "fax-lnk")
# #             company_fax_no = anchor.get_attribute('href')
# #         except (NoSuchElementException, TimeoutException):
# #             company_fax_no = ''
# #             company_fax.append(company_fax_no)
# #         finally: 
# #                 pass

#     try:
#         WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "71-lnk", "68-lnk", "66-lnk", "fax-lnk")))
#         anchor = driver.find_element(By.ID,"71-lnk")
#         company_name = anchor.text
#         company_names.append(company_name)   
#         company_website = anchor.get_attribute('href')
#     except (NoSuchElementException, TimeoutException):
#         company_names.append('')
#         company_website = ''
#     finally:
#             pass


#     driver.quit()


In [52]:
# for url in company_profile_urls:
#     driver = webdriver.Chrome(service=service)
#     driver.get(url)
    
#     try:
# #         WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.ID, "68-lnk")))
# #         WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.ID, "66-lnk")))
# #         WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.ID, "fax-lnk")))
# #         WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.ID, "71-lnk")))

#         time.sleep(7)
        
#         # Get company name and website
#         anchor = driver.find_element(By.ID,"71-lnk")
#         company_name = anchor.text
#         company_names.append(company_name)   
#         company_website = anchor.get_attribute('href')
        
#         # Confirm website
#         anchor = driver.find_element(By.ID, "68-lnk")
#         confirm_website = anchor.get_attribute('href')
#         confirm_websites.append(confirm_website)
        
#         # Get company telephone number
#         anchor = driver.find_element(By.ID, "66-lnk")
#         company_telephone = anchor.get_attribute('href')
#         company_telephones.append(company_telephone)
        
#         # Get company fax number
#         anchor = driver.find_element(By.ID, "fax-lnk")
#         company_fax_no = anchor.get_attribute('href')
#         company_fax.append(company_fax_no)
        
#     except NoSuchElementException:
#                                     pass
    
#     except TimeoutException:
#                             pass
#     finally:
#             pass
#     driver.quit()



In [54]:
print(len(company_names))
print(len(company_telephones))
print(len(company_fax))
print(len(company_websites))
print(len(confirm_websites))

163
0
0
0
0


In [73]:
print(company_names)

['Winterberger Holzbau AG', 'Alexander-Gerbi-Wellness AG', 'Föhn Transport GmbH', 'Abdichtungsbau Durrer AG', 'LBG Sursee Genossenschaft, Schenkon', 'KOCH Group AG St. Gallen', 'Felix Enzler Unterlagsböden AG', 'Auto-Center Küng AG', 'F.Tobler AG', 'Zysset + Partner AG', 'Peter Kämpfer GmbH', 'Ancotech AG', 'VIP Versandservice GmbH', 'DS-Arvotech AG', 'Bäck.Tea-Room + Garni Imseng', 'Netprom GmbH', 'Kurt Hofer Immobiliare Sagl', 'Auto Amrein AG, Altbüron', 'Fredi Sidler Transport AG', 'Gemobau Architekten AG', 'Facetec S.A.', 'Schröder Baumanagement GmbH', 'Phoenix Basel AG', 'Dobler Druck', 'Forster Plan AG', 'Caméléon Rouvinet Sàrl', 'Parkettgeschäft H.Krenz', 'Tedi Kriens AG', 'Mattiello Gerüstbau AG', 'Z-Audio Animatec AG', 'ZIP Jean-Paul Vitek', 'Paul Sauter AG', 'Rudolf Grogg AG', 'M. Bleiker Carrosserie', 'Viglezio Pneumatici SA', 'Ardüser Bauunternehmung AG', 'Norma Reiden AG', 'Garten Bieri AG', 'Jules Steiger AG', 'Ruwex AG', 'Inauen Holzbau AG', 'Passeraub Werbe AG', 'Jung &

In [72]:
url = 'https://wirmarket.wir.ch/de/companyProfile/profile/4F9AD522575A2205E054A0369F14B95F/info/?promo=false'
driver = webdriver.Chrome(service=service)
driver.get(url)
time.sleep(15)
uls = driver.find_elements(By.CLASS_NAME, "default")
list_items = driver.find_elements(By.TAG_NAME,"li")
headings = driver.find_elements(By.CLASS_NAME, "inline-block")
li_list = []
h3_list = []

for li in list_items:
    li_list.append(li.text)
  
    
#     for h3 in h3_list:
#         h3_list.append(h3.text)
    
# print(li_list)
# 
print(len(headings))
for heading in headings:
    h3_list.append(heading.text)
print(h3_list)
print(li_list)

5
['Werner Portmann', '', '', '', '']
['Home', 'WIR-Teilnehmer', 'Shop', 'Pinnwand', 'Ausschreibungen', 'Events', '', '', '', '', '', '', '', '', '', 'Firmeninformationen', 'Artikel im Shop', 'Inserate', 'Ausschreibungen', 'Bewertungen', 'Sonstige lngenieurbüros', 'wir.ch', 'Medien und Presse', 'WIR-Blog', 'Offene Stellen', 'Rechtliche Hinweise', 'Welcome-Screens', 'FAQ', 'Co-Browsing', 'Shop', 'Pinnwand', 'Ausschreibungen', 'WIR-Teilnehmer', 'Profil', 'WIR Bank Genossenschaft\nAuberg 1\n4002 Basel', '0800 947 944', 'wirmarket@wir.ch', 'Mo – Fr 07:30 – 18:00']
